In [ ]:
# Imports go here
import sys
import io
import warnings
import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=FutureWarning)
    import pandas as pd
    import numpy as np
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense
    from tensorflow.keras.layers import Dropout
    from tensorflow.keras.layers import Flatten
    from tensorflow.keras.layers import Conv2D
    from tensorflow.keras.layers import MaxPooling2D
    from tensorflow.keras.utils import to_categorical
    from tensorflow.config import list_physical_devices
    from tensorflow.config.experimental import set_memory_growth
    from sklearn import metrics
    from sklearn.model_selection import train_test_split
    import matplotlib.pyplot as plt

# from tensorflow.keras import mixed_precision
# mixed_precision.set_global_policy('mixed_float16')

class_names = ['0','1','2','3','4','5','6','7','8','9','A','B','C','D','E','F','G','H','I','J','K','L',
'M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z','a','b','d','e','f','g','h','n','q','r','t']

In [ ]:
# limit the GPU memory growth
for dev in list_physical_devices('GPU'):
    set_memory_growth(dev, True)

In [ ]:
def get_data():
    data = pd.read_csv('emnist_balanced.csv', header=None)
    return data

def prep_data(data):
    # Split first column off as label we're predicting
    X = data.iloc[:,1:]
    y = data.iloc[:,0]
    train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.5, random_state=42)

    print(train_x.shape)
    
    # Images need to be rotated before training
    train_x = np.asarray(train_x)
    test_x = np.asarray(test_x)
    train_x = np.apply_along_axis(rotate, 1, train_x)
    test_x = np.apply_along_axis(rotate, 1, test_x)

    # Normalization of grey-scale values
    train_x = train_x.astype('float32') / 255
    test_x = test_x.astype('float32') / 255

    # One-hot encoding of labels
    classes = y.unique()
    train_y = to_categorical(train_y, classes.shape[0])
    test_y = to_categorical(test_y, classes.shape[0])

    # Reshaping to 28x28x1 image of grey-scale values for CNN
    train_x = train_x.reshape(train_x.shape[0], 28, 28, 1)
    test_x = test_x.reshape(test_x.shape[0], 28, 28, 1)

    return train_x, train_y, test_x, test_y, classes

def rotate(image):
    image = image.reshape([28, 28])
    image = np.fliplr(image)
    image = np.rot90(image)
    return image

data = get_data()

train_x, train_y, test_x, test_y, classes = prep_data(data)

In [ ]:
model = Sequential()
model.add(Conv2D(filters=128, kernel_size=(5,5), padding = 'same', activation='relu', input_shape=(28, 28,1)))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Conv2D(filters=64, kernel_size=(3,3) , padding = 'same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(units=128, activation='relu'))
model.add(Dropout(.5))
model.add(Dense(units=classes.shape[0], activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
cnn = model.fit(train_x, train_y, validation_data=(test_x, test_y), epochs=5)

In [ ]:
model_json = model.to_json()
with open('model.json', 'w') as json_file:
    json_file.write(model_json)
model.save_weights('model.h5')

In [ ]:
# Plot the Loss Function
plt.plot(cnn.history['loss'], label='train loss')
plt.plot(cnn.history['val_loss'], label='val loss')
plt.legend()
plt.title('loss')
plt.show()

plt.clf()

# Plot the Accuracy
plt.plot(cnn.history['accuracy'], label='train acc')
plt.plot(cnn.history['val_accuracy'], label='val acc')
plt.legend()
plt.title('accuracy')
plt.show()